In [3]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import seaborn as sns
import time

from nltk import Counter
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahat\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
data = pd.read_csv('baby_products_key_data_2.tsv')

In [5]:
data

,Unnamed: 0,star_rating,tokenized_review_body
0,0,5.0,"['loved', 'swaddle', 'blankets', 'daughter', '..."
1,1,5.0,"['these', 'adorable', 'pacifier', 'clips', 'sa..."
2,2,5.0,"['great', 'gift']"
3,3,5.0,"['these', 'forks', 'great', 'month', 'old', 'd..."
4,4,5.0,"['i', 'wanted', 'something', 'piece', 'mind', ..."
...,...,...,...
1748943,1749143,5.0,"['my', 'month', 'old', 'son', 'loves', 'play',..."
1748944,1749144,5.0,"['not', 'son', 'love', 'toy', 'the', 'base', '..."
1748945,1749145,5.0,"['this', 'toy', 'offers', 'fun', 'learning', '..."
1748946,1749146,5.0,"['our', 'fun', 'soon', 'got', 'it', 'continues..."


In [16]:
X = data['tokenized_review_body']
y = data['star_rating']

In [6]:
def predict_text(text, model):
    word_seq = np.array(
        [
            vocab[preprocess_string(word)]
            for word in text.split()
            if preprocess_string(word) in vocab.keys()
        ]
    )

    word_seq = np.expand_dims(word_seq, axis=0)

    pad = torch.from_numpy(padding_(word_seq, 500))

    inputs = pad.to(device)

    batch_size = 1

    h = model.init_hidden(batch_size)
    h = tuple([each.data for each in h])

    output, h = model(inputs, h)

    return output.item()

In [11]:
def preprocess_string(s):
    # Remove all non-word characters (everything except numbers and letters).

    s = re.sub(r"[^\w\s]", "", s)

    # Replace all runs of whitespaces with no space.

    s = re.sub(r"\s+", "", s)

    # Replace digits with no space.

    s = re.sub(r"\d", "", s)

    return s

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [18]:
def tokenize(X_train, y_train, x_val, y_val):
    word_list = []

    stop_words = set(stopwords.words("english"))

    for sent in X_train:
        for word in sent.lower().split():
            word = preprocess_string(word)
            if word not in stop_words and word != "":
                word_list.append(word)

    # corpus: A collection of words.

    corpus = Counter(word_list)

    # Sorting on the basis of most common words.

    corpus_ = sorted(corpus, key=corpus.get, reverse=True)[:1000]

    # Create a dictionary.

    onehot_dict = {w: i + 1 for i, w in enumerate(corpus_)}

    # Tokenize.

    final_list_train, final_list_test = [], []

    for sent in X_train:
        final_list_train.append(
            [
                onehot_dict[preprocess_string(word)]
                for word in sent.lower().split()
                if preprocess_string(word) in onehot_dict.keys()
            ]
        )

    for sent in x_val:
        final_list_test.append(
            [
                onehot_dict[preprocess_string(word)]
                for word in sent.lower().split()
                if preprocess_string(word) in onehot_dict.keys()
            ]
        )

    encoded_train = [1 if label == "positive" else 0 for label in y_train]

    encoded_test = [1 if label == "positive" else 0 for label in y_val]

    return (
        np.array(final_list_train, dtype=object),
        np.array(encoded_train),
        np.array(final_list_test, dtype=object),
        np.array(encoded_test),
        onehot_dict,
    )

In [19]:
X_train, y_train, X_test, y_test, vocab = tokenize(X_train, y_train, X_test, y_test)

In [12]:
good_words = 'great'

In [13]:
test = predict_text(good_words, data)

NameError: name 'vocab' is not defined